## Set up + Imports 

In [41]:
import setup

setup.main()
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black
import yaml
import torch

Working directory:  /home/facosta/neurometry/neurometry
Directory added to path:  /home/facosta/neurometry
Directory added to path:  /home/facosta/neurometry/neurometry
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


Specify run name

In [ ]:
run_name = "run_tus9d935_s_0=1_sigma_saliency=0.05_x_saliency=0.5"

In [14]:
import os

base_dir = os.path.join(os.getcwd(), "neuroai/piRNNs/models")
configs_dir = os.path.join(base_dir, "results/configs")
models_dir = os.path.join(base_dir, "results/trained_models")

In [11]:
def _load_expt_config(run_name, configs_dir):
    config_file = os.path.join(configs_dir, f"{run_name}.json")

    with open(config_file) as file:
        return yaml.safe_load(file)

### Load experiment config

In [12]:
expt_config = _load_expt_config(run_name, configs_dir)

In [38]:
import ml_collections


def _d(**kwargs):
    """Helper of creating a config dict."""
    return ml_collections.ConfigDict(initial_dictionary=kwargs)


import ml_collections


def _convert_config(normal_config):
    """Convert a normal dictionary to ml_collections.ConfigDict.

    Parameters
    ----------
    normal_config : dict
        Configuration dictionary.

    Returns
    -------
    ml_collections.ConfigDict
        Converted configuration dictionary.

    """
    config = ml_collections.ConfigDict()

    # Training config
    config.train = {
        "load_pretrain": normal_config["load_pretrain"],
        "pretrain_path": normal_config["pretrain_path"],
        "num_steps_train": normal_config["num_steps_train"],
        "lr": normal_config["lr"],
        "lr_decay_from": normal_config["lr_decay_from"],
        "steps_per_logging": normal_config["steps_per_logging"],
        "steps_per_large_logging": normal_config["steps_per_large_logging"],
        "steps_per_integration": normal_config["steps_per_integration"],
        "norm_v": normal_config["norm_v"],
        "positive_v": normal_config["positive_v"],
        "positive_u": normal_config["positive_u"],
        "optimizer_type": normal_config["optimizer_type"],
    }

    # Simulated data config
    config.data = {
        "max_dr_trans": normal_config["max_dr_trans"],
        "max_dr_isometry": normal_config["max_dr_isometry"],
        "batch_size": normal_config["batch_size"],
        "sigma_data": normal_config["sigma_data"],
        "add_dx_0": normal_config["add_dx_0"],
        "small_int": normal_config["small_int"],
    }

    # Model parameter config
    config.model = {
        "freeze_decoder": normal_config.get("freeze_decoder", False),
        "trans_type": normal_config["trans_type"],
        "rnn_step": normal_config["rnn_step"],
        "num_grid": normal_config["num_grid"],
        "num_neurons": normal_config["num_neurons"],
        "block_size": normal_config["block_size"],
        "sigma": normal_config["sigma"],
        "w_kernel": normal_config["w_kernel"],
        "w_trans": normal_config["w_trans"],
        "w_isometry": normal_config["w_isometry"],
        "w_reg_u": normal_config["w_reg_u"],
        "reg_decay_until": normal_config["reg_decay_until"],
        "adaptive_dr": normal_config["adaptive_dr"],
        "s_0": normal_config["s_0"],
        "x_saliency": normal_config["x_saliency"],
        "sigma_saliency": normal_config["sigma_saliency"],
        "reward_step": normal_config["reward_step"],
        "saliency_type": normal_config["saliency_type"],
    }

    # Path integration config
    config.integration = {
        "n_inte_step": normal_config["n_inte_step"],
        "n_traj": normal_config["n_traj"],
        "n_inte_step_vis": normal_config["n_inte_step_vis"],
        "n_traj_vis": normal_config["n_traj_vis"],
    }

    return config

### Load Trained Model

In [39]:
from neurometry.neuroai.piRNNs.models import model

config = _convert_config(expt_config)
model_config = model.GridCellConfig(**config.model)
device = "cuda"
model = model.GridCell(model_config).to(device)

In [44]:
trained_model_path = os.path.join(models_dir, f"{run_name}_model.pt")
trained_model = torch.load(trained_model_path, map_location=device)
model.load_state_dict(trained_model["state_dict"])

<All keys matched successfully>

### Load evaluation data (trajectories)

In [46]:
from neurometry.neuroai.piRNNs.models import input_pipeline

In [95]:
import numpy as np
import neurometry.neuroai.piRNNs.models.utils as utils

rng = np.random.default_rng()

eval_dataset = input_pipeline.EvalDataset(
    rng, config.integration, config.data.max_dr_trans, config.model.num_grid
)

eval_iter = iter(eval_dataset)

trajectories = next(eval_iter)["traj"]["traj"]

In [96]:
eval_data = utils.dict_to_device(next(eval_iter), device)

In [92]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Determine plot limits based on trajectory data
max_x = np.max(trajectories[:, :, 0]) + 1
max_y = np.max(trajectories[:, :, 1]) + 1


def animate(i, traj_idx):
    plt.cla()  # Clear current plot
    traj = trajectories[traj_idx]
    plt.plot(
        traj[:i, 0], traj[:i, 1], "r-", alpha=0.3
    )  # Plot trail with reduced opacity
    plt.plot(traj[i, 0], traj[i, 1], "b.", markersize=15)  # Plot current point
    plt.xlim(-max_x, max_x)  # Adjust x-axis limits as needed
    plt.ylim(-max_y, max_y)  # Adjust y-axis limits as needed
    plt.title(f"Time Step {i}")  # Set title for the frame
    plt.xlabel("X Axis")
    plt.ylabel("Y Axis")
    # plt.axis("equal")


# Set up figure and animation
fig = plt.figure()
ani = animation.FuncAnimation(
    fig, animate, frames=num_steps_per_trajectory, fargs=(0,), interval=200
)

# Display animation inline in Jupyter Notebook
%matplotlib notebook
HTML(ani.to_html5_video())

<IPython.core.display.Javascript object>